In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import  BernoulliNB, GaussianNB, MultinomialNB

In [27]:
data = pd.read_csv("~/Desktop/mcc20192/mlearning/tareas/tarea2/nb_data/breast-cancer-wisconsin.data")
data

# Preparation of data
#Partición aleatoria del conjunto de datos
train_data, test_data= train_test_split(data, test_size=0.30, random_state=0)
print(len(train_data),len(test_data))
print(train_data.index)

489 210
Int64Index([285,  71, 519, 407, 603, 302, 104, 631, 188, 516,
            ...
            472,  70, 599, 277,   9, 359, 192, 629, 559, 684],
           dtype='int64', length=489)


In [28]:
def vector_of_experimental_means(df):
    means = np.zeros(9)
    for i in range(1,10):
        known_count = 0
        s = 0
        j = 0
        for k in df.index:
            if df.loc[k][i] != '?':
                s += int(df.loc[k][i])
                known_count += 1
                j += 1
        means[i-1] = s/known_count
    return means

means = vector_of_experimental_means(data)
print(means)

[4.41773963 3.13447783 3.2074392  2.80686695 3.21602289 3.54465593
 3.43776824 2.86695279 1.58941345]


In [29]:
def get_labels(df):
    df_labels = np.zeros(len(df))
    i = 0
    for k in df.index:
        df_labels[i] = df.loc[k][-1] 
        i+= 1
    return df_labels

# Matriz cuyas entradas desconocidas son reemplazadas por la media experimental del atributo al que corresponden
def get_matrix(df):
    df_matrix = np.zeros(shape=(len(df),9))
    i = 0
    for k in df.index:
        for j in range(1,len(df.loc[k])-1):
            if df.loc[k][j] != '?':
                df_matrix[i][j-1] = df.loc[k][j]
            else:
                df_matrix[i][j-1] = means[j-1]
        i += 1
    return df_matrix

# Preparación del conjunto de entrenamiento
train_labels = get_labels(train_data)
train_matrix = get_matrix(train_data)
print(len(train_matrix),len(train_labels))

#print(train_data,train_matrix,train_labels)

# Preparación del conjunto de validación
test_labels = get_labels(test_data)
test_matrix = get_matrix(test_data)
print(len(test_matrix),len(test_labels))
print('ok')

489 489
210 210
ok


In [30]:
# Proponemos los siguientes clasificadores
bernoulli_classifier = BernoulliNB()
normal_classifier = GaussianNB()
multinomial_classifier = MultinomialNB()

# Entrenamiento
print(bernoulli_classifier.fit(train_matrix,train_labels))
print(normal_classifier.fit(train_matrix,train_labels))
print(multinomial_classifier.fit(train_matrix,train_labels))

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
GaussianNB(priors=None, var_smoothing=1e-09)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


In [31]:
# Pruebas en el conjunto de validación
test_bernoulli = bernoulli_classifier.predict(test_matrix)
test_normal = normal_classifier.predict(test_matrix)
test_multinomial = multinomial_classifier.predict(test_matrix)

# Pruebas en el conjunto de validación
train_bernoulli = bernoulli_classifier.predict(train_matrix)
train_normal = normal_classifier.predict(train_matrix)
train_multinomial = multinomial_classifier.predict(train_matrix)

def classification_errors(predicted,real):
    s = 0
    for i in range(0,len(predicted)):
        if predicted[i] != real[i]:
            s += 1
    return s

print('Sobre el conjunto de validación')    
print(classification_errors(test_bernoulli,test_labels))
print(classification_errors(test_normal,test_labels))
print(classification_errors(test_multinomial,test_labels))

print('Sobre el conjunto de entrenamiento')
print(classification_errors(train_bernoulli,train_labels))
print(classification_errors(train_normal,train_labels))
print(classification_errors(train_multinomial,train_labels))

Sobre el conjunto de validación
75
13
28
Sobre el conjunto de entrenamiento
166
13
51
